# Pokedex

imports

In [102]:
import os
import cv2
import glob
import sklearn
import numpy
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split
import random
from sklearn.ensemble import RandomForestClassifier

parametros globais

In [103]:
# parametros globais
raiz_dataSet = './images/'

Pre processamentos

In [104]:

def resize(im, par):
    im = cv2.resize(im, (par[0], par[0]))
    return im


def saturation(im, saturation_factor):
    im[:, :, 1] = numpy.clip(
        im[:, :, 1] * saturation_factor, 0, 255).astype(numpy.uint8)
    return im


def brightness(im, brightness_factor):
    im[:, :, 2] = numpy.clip(
        im[:, :, 2] * brightness_factor, 0, 255).astype(numpy.uint8)
    return im


def gaussian(im, par):
    im = cv2.GaussianBlur(im, (par[0], par[0]), par[1])
    return im


def median(im, par):
    im = cv2.medianBlur(im, par[0])
    return im


def bilateral(im, par):
    im = cv2.bilateralFilter(im, par[0], par[0]*1, par[0]/2)
    return im


def gamma(im, par):
    invGamma = 1.0 / par[0]
    table = numpy.array([((i / 255.0) ** invGamma) * 255
                         for i in numpy.arange(0, 256)]).astype("uint8")
    return cv2.LUT(im, table)


def equalize(im, par):
    return cv2.equalizeHist(im)


'''
'log' : {
      'value' : [0]
    }
'''


def logfilter(im, par):
    m = numpy.max(im)
    if m == 0:
        m = 255
    c = 255 / numpy.log(1 + m)
    log_image = c * (numpy.log(im + 1))
    log_image = numpy.array(log_image, dtype=numpy.uint8)
    return log_image


'''
'clahe' : {
      'clipLimit' : [2, 4, 6],
      'size' : [8, 12, 16]
    }
'''


def clahe(im, par):
    clahe = cv2.createCLAHE(clipLimit=par[0], tileGridSize=(par[1], par[1]))
    cl1 = clahe.apply(im)
    return cl1

Otimização

In [105]:
# um hog básico
def compute_hog(im):
    winSize = im.shape
    blockSize = (int(winSize[0]/blocks), int(winSize[1]/blocks))
    blockStride = blockSize
    cellSize = (int(blockSize[0]/2), int(blockSize[1]/2))
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    vetor = hog.compute(im)
    return numpy.reshape(vetor, (vetor.shape[0]))


# parâmetros HOG
blocks = 8
nbins = 24

In [106]:
import cv2
import numpy as np


def calcular_media_de_cores(im):
    # if im.shape[2] == 4:
    #     im = cv2.cvtColor(im, cv2.COLOR_BGRA2BGR)

    # # Inicializa as médias de cada canal com zero
    # media_r = 0
    # media_g = 0
    # media_b = 0

    # # Inicializa os contadores para cada canal
    # contador_r = 0
    # contador_g = 0
    # contador_b = 0

    # for i in range(im.shape[0]):
    #     for j in range(im.shape[1]):
    #         pixel = im[i][j]

    #         # Verifica se o pixel não é preto (não tem valor zero)
    #         if np.any(pixel != 0):
    #             if pixel[0] != 0:
    #                 contador_r += 1
    #                 media_r += pixel[0]
    #             if pixel[1] != 0:
    #                 contador_g += 1
    #                 media_g += pixel[1]
    #             if pixel[2] != 0:
    #                 contador_b += 1
    #                 media_b += pixel[2]

    # # Calcula a média de cada canal
    # if contador_r != 0:
    #     media_r /= contador_r
    # if contador_g != 0:
    #     media_g /= contador_g
    # if contador_b != 0:
    #     media_b /= contador_b
    max_valor = np.max(im)
    # print(max_valor)

    if im.shape[2] == 4:
        im = cv2.cvtColor(im, cv2.COLOR_BGRA2RGB)
    else:
        img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    # Calcular a média das intensidades de cada canal de cor (R, G, B)
    media_r_normalizada = np.mean(img_rgb[:, :, 0])/max_valor
    media_g_normalizada = np.mean(img_rgb[:, :, 1])/max_valor
    media_b_normalizada = np.mean(img_rgb[:, :, 2])/max_valor

    return [media_r_normalizada, media_g_normalizada, media_b_normalizada]


# Exemplo de uso:
imagem_exemplo = cv2.imread('./images/images/skiploom.png')
medias_de_cores = calcular_media_de_cores(imagem_exemplo)
print(medias_de_cores)

[0.04950190631808279, 0.06610021786492375, 0.02270561002178649]


In [107]:
# defina as funções de otimização para serem avaliadas
parametros = {
    'saturation': {
        'value': [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
    },
    'brightness': {
        'value': [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6]
    },
    'clahe': {
        'clipLimit': [4, 2],
        'size': [16, 12, 8]
    },
    'equalize': {
        'value': [0]
    },
    'gamma': {
        'value': [0.5, 1, 1.5, 2]
    },
    'median': {
        'filter': [3, 5, 7]
    },
    'resize': {
        'size': [128, 256, 512]
    },
    'gaussian': {
        'filter': [3, 5, 7],
        'desv': [0, 0.5, 1, 1.5]
    }


}

# quando se deseja encontrar apenas uma função de pré-processamento


def evaluateGridOne(parametros):
    # print('Labels: ')
    # carrega a base

    df = pd.read_csv(raiz_dataSet+'pokemon.csv', index_col='Name')
    df = df.drop(['Type2'], axis=1)
    best_size = 128  # vai ser usado como padrão depois para os demais métodos
    best_score = 0
    best_param = []

    # 1: para cada algoritmos de pre-processamento
    for alg, pars in parametros.items():
        print("Alg:" + alg, end=" ")
        items = list(product(*pars.values()))
        # 2: para cada parâmetro
        for par in items:
            print("Par:" + str(par), end=" ")
            data = []
            labels = []
            dataset = glob.glob(raiz_dataSet + "images/*")  # + image_format
            for arq in dataset:
                # pega a imagem
                im = cv2.imread(arq, 0)
                # 3: pre processa
                if alg == 'resize':
                    im = resize(im, par)
                elif alg == 'gaussian':
                    im = cv2.resize(im, (best_size, best_size))
                    im = gaussian(im, par)
                elif alg == 'brightness':
                    im = cv2.resize(im, (best_size, best_size))
                    im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
                    im = brightness(im, par)
                    im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                elif alg == 'saturation':
                    im = cv2.resize(im, (best_size, best_size))
                    im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
                    im = saturation(im, par)
                    im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                elif alg == 'median':
                    im = cv2.resize(im, (best_size, best_size))
                    im = median(im, par)
                elif alg == 'gamma':
                    im = cv2.resize(im, (best_size, best_size))
                    im = gamma(im, par)
                elif alg == 'equalize':
                    im = cv2.resize(im, (best_size, best_size))
                    im = equalize(im, par)
                elif alg == 'clahe':
                    im = cv2.resize(im, (best_size, best_size))
                    im = clahe(im, par)

                # computa as características
                data.append(compute_hog(im))
                nome = arq[16:]
                nome = nome.split('.')
                # adiciona as labels
                labels.append(df.loc[nome[0]]['Type1'])

            # conjunto completo
            X = numpy.asarray(data)
            y = numpy.asarray(labels)

            # 4: estima o resultado
            # sempre o mesmo aleatório para dar sempre a mesma base
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, random_state=20, train_size=0.8)

            # o classificador é apenas para realizar um fitness do problema
            clf = RandomForestClassifier(
                n_estimators=100, max_depth=3, random_state=random.randrange(1, 1000)).fit(X_train, y_train)
            y_get = clf.predict(X_test)
            score = sklearn.metrics.balanced_accuracy_score(y_test, y_get)
            print("Balanced Acc: " + str(score), end="\n")
            if score > best_score:
                best_score = score
                best_param = par
                if alg == 'resize':
                    best_size = par[0]

        print("Best param alg: " + alg + ": " + str(best_param))
        best_param = []
        best_score = 0


evaluateGridOne(parametros)

Alg:saturation Par:(0.5,) Balanced Acc: 0.061764705882352944
Par:(1,) Balanced Acc: 0.057058823529411766
Par:(1.5,) Balanced Acc: 0.07647058823529412
Par:(2,) Balanced Acc: 0.07411764705882352
Par:(2.5,) Balanced Acc: 0.05647058823529412
Par:(3,) Balanced Acc: 0.058823529411764705
Par:(3.5,) Balanced Acc: 0.0688235294117647
Par:(4,) Balanced Acc: 0.06955882352941177
Par:(4.5,) Balanced Acc: 0.06764705882352941
Par:(5,) Balanced Acc: 0.0688235294117647
Best param alg: saturation: (1.5,)
Alg:brightness Par:(0.2,) Balanced Acc: 0.05941176470588235
Par:(0.4,) Balanced Acc: 0.07117647058823529
Par:(0.6,) Balanced Acc: 0.06411764705882352
Par:(0.8,) Balanced Acc: 0.07176470588235294
Par:(1,) Balanced Acc: 0.06823529411764706
Par:(1.2,) Balanced Acc: 0.0611764705882353
Par:(1.4,) Balanced Acc: 0.05235294117647059
Par:(1.6,) Balanced Acc: 0.06705882352941177
Best param alg: brightness: (0.8,)
Alg:clahe Par:(4, 16) Balanced Acc: 0.06235294117647059
Par:(4, 12) Balanced Acc: 0.0611764705882353
P

KeyboardInterrupt: 

In [ ]:
from rembg import remove
from PIL import Image
dataset = glob.glob(raiz_dataSet + "images/*")
for arq in dataset:
    if (arq.split('.')[2] == 'jpg'):
        entrada_path = arq
        saida = arq.split('.')[0] + arq.split('.')[1]
        saida_path = '.' + saida + '.png'
        entrada = Image.open(entrada_path)  # carrega a imagem
        saida = remove(entrada)  # função que retira o fundo
        saida.save(saida_path)
        os.remove(arq)

In [ ]:
from itertools import permutations

melhores_parametros = {
    'saturation': {
        'value': [3.5]
    },
    'brightness': {
        'value': [0.4]
    },
    'gaussian': {
        'filter': [5],
        'desv': [0.5]
    },
    'median': {
        'filter': [7]
    },
    'gamma': {
        'value': [1.5]
    },
    'equalize': {
        'value': [0]
    },
    'clahe': {
        'clipLimit': [4],
        'size': [16]
    }
}

# quando se deseja encontrar mais de uma função "nstages" de otimização que devem ser aplicadas em Pipeline


def evaluateGridStage(parametros, nstages):
    df = pd.read_csv(raiz_dataSet+'pokemon.csv', index_col='Name')
    df = df.drop(['Type2'], axis=1)
    best_size = 256

    best_score = 0
    best_param = []
    contador = 1

    items = list(permutations(parametros, nstages))

    # 1: para cada combinação de métodos
    for combo in items:
        print("Permutação (" + str(contador) + " de " +
              str(len(items)) + "):" + str(combo), end=" ")
        contador = contador + 1
        data = []
        labels = []
        dataset = glob.glob(raiz_dataSet + "images/*")
        for arq in dataset:
            im = cv2.imread(arq, 0)
            im = cv2.resize(im, (best_size, best_size))

            # 2: pre processa
            for item in combo:
                p = (list(melhores_parametros[item].values()))
                if item == 'gaussian':
                    im = gaussian(im, [p[0][0], p[1][0]])
                elif item == 'saturation':
                    im = saturation(im, [p[0][0]])
                elif item == 'brightness':
                    im = brightness(im, [p[0][0]])
                elif item == 'median':
                    im = median(im, [p[0][0]])
                elif item == 'gamma':
                    im = gamma(im, [p[0][0]])
                elif item == 'equalize':
                    im = equalize(im, '')
                elif item == 'clahe':
                    im = clahe(im, [p[0][0], p[1][0]])
            # computa as características
            data.append(compute_hog(im))
            nome = arq[16:]
            nome = nome.split('.')
            # adiciona as labels
            labels.append(df.loc[nome[0]]['Type1'])
        # conjunto completo
        X = numpy.asarray(data)
        y = numpy.asarray(labels)

        # 3: estima o resultado
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=0.8, random_state=100)

        clf = RandomForestClassifier(
            n_estimators=100, max_depth=3, random_state=100).fit(X_train, y_train)
        y_get = clf.predict(X_test)
        score = sklearn.metrics.balanced_accuracy_score(y_test, y_get)
        print("Balanced Acc: " + str(score), end="\n")
        if score > best_score:
            best_score = score
            best_param = combo

    print("Best param combo: " + str(best_param))
    best_param = []
    best_score = 0


evaluateGridStage(melhores_parametros, 4)

Permutação (1 de 840):('saturation', 'brightness', 'gaussian', 'median') Balanced Acc: 0.07672634271099744
Permutação (2 de 840):('saturation', 'brightness', 'gaussian', 'gamma') Balanced Acc: 0.0736303674787993
Permutação (3 de 840):('saturation', 'brightness', 'gaussian', 'equalize') Balanced Acc: 0.08439897698209718
Permutação (4 de 840):('saturation', 'brightness', 'gaussian', 'clahe') Balanced Acc: 0.08076457127473415
Permutação (5 de 840):('saturation', 'brightness', 'median', 'gaussian') Balanced Acc: 0.0736303674787993
Permutação (6 de 840):('saturation', 'brightness', 'median', 'gamma') Balanced Acc: 0.07672634271099744
Permutação (7 de 840):('saturation', 'brightness', 'median', 'equalize') Balanced Acc: 0.07618791223583254
Permutação (8 de 840):('saturation', 'brightness', 'median', 'clahe') Balanced Acc: 0.07820702651770091
Permutação (9 de 840):('saturation', 'brightness', 'gamma', 'gaussian') Balanced Acc: 0.08641809126396553
Permutação (10 de 840):('saturation', 'brightn

In [108]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from skimage.feature import local_binary_pattern
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

dataset = glob.glob(raiz_dataSet + 'images/*')
data = []
labels = []
data_cores = []
data_lbp = []
# best_size = 128
best_size = 256
# csv
df = pd.read_csv(raiz_dataSet+'pokemon.csv', index_col='Name')
df = df.drop(['Type2'], axis=1)

n = 3
folds = 5
recall = []
precision = []
f1 = []
matriz = []

for arq in dataset:
    im = cv2.imread(arq, 0)
    im = resize(im, (best_size, best_size))
    im = equalize(im, 0)
    im = gamma(im, [1.5])
    im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
    im = brightness(im, [0.4])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = gaussian(im, (5, 0.5))

    # computa as características
    data.append(compute_hog(im))
    data_lbp.append(local_binary_pattern(im, 4*3, 3, 'uniform'))
    nome = arq[16:]
    nome = nome.split('.')
    # adiciona as labels

    labels.append(df.loc[nome[0]]['Type1'])

for arq in dataset:
    im = cv2.imread(arq)
    im = resize(im, (best_size, best_size))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = equalize(im, 0)
    im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
    im = gamma(im, [1.5])
    im = brightness(im, [0.4])
    im = gaussian(im, (5, 0.5))
    # computa as características
    data_cores.append(calcular_media_de_cores(im))


# conjunto hog
x_hog = numpy.asarray(data)
y = numpy.asarray(labels)

print(y.shape)

# conjunto cores
x_cores = numpy.asarray(data_cores)
print(x_hog.shape)

# conjunto lbp
x_lbp = numpy.asarray(data_lbp)
x_lbp = numpy.reshape(x_lbp, (808, -1))
print(x_lbp.shape)

x = numpy.concatenate((x_hog, x_lbp), axis=1)
for i in range(100):
    x = numpy.concatenate((x, x_cores), axis=1)

for i in range(n):
    scv = StratifiedKFold(n_splits=folds, shuffle=True)

    scv.get_n_splits(x, y)
    pred_folds = []

    for train_index, test_index in scv.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]

    # print('SVC: ')
    # svm_model = SVC(kernel='linear', C=100)
    # svm_model.fit(x_train, y_train)
    # y_get = svm_model.predict(x_test)

    print('Random Forest: ')
    clf = RandomForestClassifier(
        n_estimators=1500, max_depth=1000).fit(x_train, y_train)
    y_get = clf.predict(x_test)

    pred_folds.append(y_get)

    y_get = np.concatenate(numpy.asarray(pred_folds)[:])

    matriz.append(confusion_matrix(y_test, y_get))
    # média: ponderada
    recall.append(recall_score(y_test, y_get, average='weighted'))
    precision.append(precision_score(y_test, y_get, average='weighted'))
    f1.append(f1_score(y_test, y_get, average='weighted'))

# calculando as médias das execuções
recall_media = np.asarray(recall).mean(axis=0)
prec_media = np.asarray(precision).mean(axis=0)
f1_media = np.asarray(f1).mean(axis=0)

# calculando os desvios das execuções
recall_std = np.asarray(recall).std(axis=0)
prec_std = np.asarray(precision).std(axis=0)
f1_std = np.asarray(f1).std(axis=0)

print("Recall: " + str(recall_media) + " +- " + str(recall_std))
print("Precisão: " + str(prec_media) + " +- " + str(prec_std))
print("F1: " + str(f1_media) + " +- " + str(f1_std))


print(sklearn.metrics.confusion_matrix(y_test, y_get))

(808,)
(808, 6144)
(808, 65536)


/home/igor/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Random Forest: 
